## Import Libraries

In [29]:
import pandas as pd

## Load Data
1. Load Data of BTC-USD from `2015` to `2022` from `BTC-USD-2015to2022.csv`
2. Set Parameter `initAsset` for initial asset.

In [30]:
btcDf= pd.read_csv('BTC-USD-2015to2022.csv')
initialAsset = 1000

## Functions

In [31]:
btc = 0
weeknum = 0
def Tade1(row):
    global initAsset
    global btc
    #Buy on Sunday
    if row['weekDay'] == 'Sunday':
        btc = initAsset / row['Open']
        row['Btc'], row['Usd'] = btc, 0
    # Sell on Wednesday
    elif row['weekDay'] == 'Wednesday':
        initAsset = btc * row['Close']
        row['Btc'], row['Usd'] = 0, initAsset
    return row

def Tade2(row):
    global initAsset, btc, weeknum
    #Buy
    btc = initAsset / row['sunOpen']
    # Sell
    initAsset = btc * row['wedClose']
    weeknum += 1
    row['Btc'], row['Usd'], row['weekNum'] = btc, initAsset, f'Week {weeknum}'
    return row

def Trade3(row):
    global initAsset, weeknum
    #Buy and Sell
    initAsset = initAsset / row['sunOpen'] * row['wedClose']
    weeknum += 1
    row['Asset'], row['weekNum'], row['rowNum'] = initAsset, f'Week {weeknum}', weeknum
    return row


## Method 1
Keep the Sunday and Wednesday rows and trade on them buying on Sunday and selling on Wednesday.

In [32]:
initAsset = initialAsset
df = pd.DataFrame(btcDf)
# include the columns you want
# df['date'] = df.index
df['Date'] = pd.to_datetime(df['Date'])
df['weekDay'] = df['Date'].dt.day_name()
df = df[df.columns[df.columns.isin(['Open', 'Close', 'weekDay'])]]
df['Usd'] = 0
df['Btc'] = 0
df = df[(df['weekDay'] == 'Sunday') | (df['weekDay'] == 'Wednesday')]

df = df.apply(Tade1, axis=1)
denefit = df.loc[df['weekDay'] == 'Wednesday', 'Usd'].values[-1] - 1000
print(f'Benefit: {denefit:.2f} USD')

Benefit: 6134.36 USD


## Method 2
Keep the Sunday and Wednesday rows and trade on them buying on Sunday and selling on Wednesday.
The Sunday and Wednesday rows are kept, column Close one row shifted back as Close_Wednesday - shift(-1) - and Wednesday rows are deleted then traded on them, buying and selling separately on Sunday.

In [33]:
initAsset = initialAsset
df = pd.DataFrame(btcDf)
# include the columns you want
df['Date'] = pd.to_datetime(df['Date'])
df['weekDay'] = df['Date'].dt.day_name()
df = df[df.columns[df.columns.isin(['Open', 'Close', 'weekDay'])]]
df = df[(df['weekDay'] == 'Sunday') | (df['weekDay'] == 'Wednesday')]
df['wedClose'] = df['Close'].shift(-1)
df['Usd'] = 0
df['Btc'] = 0
df = df[(df['weekDay'] == 'Sunday')]
df = df.rename(columns={"weekDay": "weekNum", "Open": "sunOpen", "Close": "sunClose"})

df = df.apply(Tade2, axis=1)
benefit = df['Usd'].values[-1] - 1000
print(f'Benefit: {benefit:.2f} USD')

Benefit: 6134.36 USD


## Method 3
Keep the Sunday and Wednesday rows, column Close one row shifted back as `Close_Wednesday` - shift(-1) - then delete Wednesday's rows and traded on remind rows (Sunday's Rows).

In [34]:
initAsset = initialAsset
df = pd.DataFrame(btcDf)
# include the columns you want
df['Date'] = pd.to_datetime(df['Date'])
df['weekDay'] = df['Date'].dt.day_name()
df = df[df.columns[df.columns.isin(['Open', 'Close', 'weekDay'])]]
df = df[(df['weekDay'] == 'Sunday') | (df['weekDay'] == 'Wednesday')]
df['wedClose'] = df['Close'].shift(-1)
df['Asset'] = 0
df = df[(df['weekDay'] == 'Sunday')]
df = df.rename(columns={"weekDay": "weekNum", "Open": "sunOpen", "Close": "sunClose"})

df = df.apply(Trade3, axis=1)
benefit = df['Asset'].values[-1] - 1000
print(f'Benefit: {benefit:.2f} USD')

Benefit: 6134.36 USD
